# Get all items by location, analyse and save to Excel

This notebook fecthes all items for a range of locations and saves slected fields to a multi-sheet Excel spreadsheet where items are distributed into separate sheets by location.

## Preparations

### Install modules
Most modules that we will use are already installed on Google Colab. The below we need to actively install.

In [ ]:
# We will use folioclient to communicate with FOLIO: https://pypi.org/project/folioclient/

!pip install folioclient

In [ ]:
# We will use xlsxwriter to create Excel files: https://pypi.org/project/XlsxWriter/ 

!pip install xlsxwriter

### Import modules
Modules that are installed need to be imported, so that we can access them in our code.

In [ ]:
import pandas
from pandas import json_normalize
from folioclient import FolioClient
from getpass import getpass
from google.colab import files

## Connect to FOLIO
We log into FOLIO by providing the okapi URL and the tenant ID, as well as the username and password of a FOLIO user. FOLIO will return a token which contains information about which user we are acting as, and what permissions that user has.

Note that tokens and passwords are sensitive data which should always be kept secret and secure.

### Provide tenant information and credentials

In [ ]:
okapi_url = input("Enter your okapi URL here: ")
tenant_id = input("Enter your tenant ID here: ")

# The getpass module lets us enter credentials at runtime
username = getpass('Enter usernme here: ')
password = getpass('Enter password here: ')


### Log into FOLIO
We will use the FolioClient module to make requets to FOLIO.

In [ ]:
# This will log us into FOLIO, and temporarily store important information like the token in the background
try:
  folio_client = FolioClient(okapi_url, tenant_id, username, password)
  print(f"Successfully logged into FOLIO as user {username}")
except Exception as e:
  print(e)

## Design the search query
This is the part where you specify what data you want to get from FOLIO.

### Endpoint, object and page size
Specify the endpoint you want to search, what objects you expect to get back, and an appropriate page size.

Appropriate page size varies by endpoint. For /inventory/items the maximum is around 100, but for /item-storage/items you can get tens of thousands of records at a time.

In [ ]:
endpoint = "/inventory/items"
record_type = "items"
page_size = "100"
query = "?query=effectiveLocationId=="

### Select locations to work with
One way to do this is to search by code/name, or recurring patterns or prefixes in the code/name.

In [ ]:
# Get all the locations you have in FOLO
all_locations = pandas.json_normalize(folio_client.locations)[["code", "name", "id"]]
print(f"Found {len(all_locations)} locations in total.")

In [ ]:
# Specify a location code prefix which identifies the locations you want to fetch
location_code_prefix = "UC/HP/SPCL/Mss"

In [ ]:
# Select the locations where the code starts with the code prefix
selected_locations = all_locations.loc[all_locations["code"].str.startswith(location_code_prefix)]
selected_locations = selected_locations.to_dict("records")
print(f"Found {len(selected_locations)} locations where the code starts with the pattern {location_code_prefix}")

## Get the item data from FOLIO
This is where you actually fetch the data from FOLIO. 

If your question is quite simple, and only requires you to interact with one API, you can do this with one query as in the example below.
* What are the titles and barcodes of all the items with this effective location?

An example of a more compelx question, which requires queries to multiple APIs, could be
* What are the names of all users who have ever borrowed a book from x location?

### Fecth all the items which match your query

In [ ]:
# Create an empty list. This is the basket where you will store retrieved items.
data = []

# For each of the locations that you have selected, do the following
for location in selected_locations:
  
  # Complete your query UUID to your query
  print(f"Fetching items with location code {location['code']}")
  query_with_uuid = query + location['id']

  # Fecth all the results from FOLIO, in batches of whatever your page size is
  for i, record in enumerate(folio_client.folio_get_all(
      path = endpoint, key = record_type, limit = int(page_size))):
    
    # Put the results in the basket 
    data.append(record)

    # Print out progress
    page_number = int(i / int(page_size))
    if (i > 0) and i % int(page_size) == 0:
        print(f"Fecthed {i} records in {page_number} pages.")

  print(f"Done! Fetched {i+1} records in {page_number + 1} pages.\n")

## Work with the data
There is a lot you can do here: create different tables based on various critera, analyse the content of the data, rename headers, change the content of the data.

### Read the item data into a dataframe and inspect it
A dataframe is a table-like structure similar to what you might be used working with in for example Excel.

In [ ]:
  # Read the json data into a dataframe
  df = pandas.json_normalize(data)

  # Inspect the headers in the dataframe
  df.info()

In [ ]:
  # Look at the first two rows in the dataframe
  df.head(2)

### Create a custom dataframe containing only the columns you want

In [ ]:
  # Create a new dataframe, stock_list, with the specified columns
  stock_list = df[["effectiveLocation.name","callNumber", "title", "materialType.name", "status.name", "status.date", "lastCheckIn.dateTime"]]

  # Inspect the first two rows of the new dataframe
  stock_list.head(2)

## Print your custom dataframe to an Excel file
You can download the file to your local computer (as below) or save it to for example Google Drive.
You can read more about file storage at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=k9q0pp33dckN

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pandas.ExcelWriter("stock_lists.xlsx", engine="xlsxwriter")

# For each unique item location, create a separate sheet in the same workbook
for location in selected_locations:
  
  # Replace slashes with dashes to comply with Excel naming rules
  formatted_location_name = location["name"].replace("/", "-")
  
  # Create a dataframe containing the items with this location
  df_per_location = stock_list.loc[stock_list["effectiveLocation.name"] == location["name"]]

  df_per_location.to_excel(
      writer, 
      sheet_name=formatted_location_name, 
      encoding = 'utf-8-sig',
      index=False)

# Close the Pandas Excel writer and download the Excel file
writer.save()

# Download the file
files.download('stock_lists.xlsx')
